In [ ]:
%load_ext autoreload
%autoreload 2
import os
os.environ['SM_FRAMEWORK'] = 'tf.keras'

In [ ]:
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
from watch_recognition.predictors import TFLiteDetector
from watch_recognition.predictors import KPPredictor
from watch_recognition.predictors import HandPredictor
from watch_recognition.predictors import RotationPredictor
keypoint_model = '../cloud_train/local-training-output/heatmap-regression_098DB017/models/keypoint/efficientnetb0-unet-96-hands/run_1636479122.611962/'
sm_model_path = "../models/effnet-b3-FPN-(160, 160)-1040-tversky/6602EDDE"

hand_predictor = HandPredictor(sm_model_path)
kp_predictor = KPPredictor(keypoint_model)
rp = RotationPredictor("./models/angle-classifier/efficientnetb0-8/run_1635014448.999021/")

In [ ]:
detector = TFLiteDetector("./models/detection/efficientdet_lite0/run_1631453124.255371/model.tflite")

In [ ]:
examples_dir = Path("../example_data")
example_images = list(examples_dir.glob("IMG*"))  + list(examples_dir.glob("Zrzut*"))
example_images

In [ ]:
from tqdm import tqdm
import dataclasses
import numpy as np
from watch_recognition.models import points_to_time
renders = []
for img_path in tqdm(example_images):
    with Image.open(img_path) as pil_img:
        frame = np.array(pil_img)
        bboxes = detector.predict(pil_img)
        new_results = []
        for box in bboxes:
            pred_center, pred_top = kp_predictor.predict_from_image_and_bbox(pil_img, box, rotation_predictor=rp)
            frame = pred_center.draw_marker(frame, thickness=2)
            frame = pred_top.draw_marker(frame, thickness=2)
            minute_and_hour, other = hand_predictor.predict_from_image_and_bbox(pil_img, box, pred_center, debug=False)
            if minute_and_hour:
                pred_minute, pred_hour = minute_and_hour
                minute_kp = dataclasses.replace(pred_minute.end, name="Minute")
                hour_kp = dataclasses.replace(pred_hour.end, name="Hour")
                read_hour, read_minute = points_to_time(
                    pred_center, hour_kp, minute_kp, pred_top
                )
                frame = pred_minute.draw(frame, thickness=3)
                frame = pred_minute.end.draw_marker(frame, thickness=2)
                frame = pred_hour.draw(frame, thickness=7, color=(255, 128,0))
                frame = pred_hour.end.draw_marker(frame, thickness=2)

                time = f"{read_hour:02.0f}:{read_minute:02.0f}"
                new_results.append(dataclasses.replace(box, name=time))
            else:
                new_results.append(dataclasses.replace(box, name="???"))

            for line in other:
                frame = line.draw(frame, thickness=2, color=(255, 0, 0))
        for box in new_results:
            frame = box.draw(frame)
        renders.append(frame)

In [ ]:
from IPython.display import display

In [ ]:
for source, render in zip(example_images, renders):
    img = Image.fromarray(render)
    save_path = source.parent / (source.stem + "_render.jpg")
    img.save(save_path)
    img .thumbnail((1000, 1000))
    display(img)